In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import differential_evolution, fmin

def clean_data_single_participant(file_path):
    est = np.zeros((60, 3))
    queryOrder = [
        " cold", " rainy", " not cold", " not rainy",
        " cold and rainy", " rainy and not cold",
        " cold and not rainy", " not cold and not rainy",
        " cold or rainy", " rainy or not cold",
        " cold or not rainy", " not cold or not rainy",
        " cold given rainy", " not cold given rainy",
        " cold given not rainy", " not cold given not rainy",
        " rainy given cold", " not rainy given cold",
        " rainy given not cold", " not rainy given not cold"
    ]

    df = pd.read_csv('/content/drive/MyDrive/all_data/PrEstExp_811_111418_122039.csv')
    for j, q in enumerate(queryOrder):
        nowEst = df[df['querydetail'] == q]['estimate']
        nowEstValues = nowEst.values / 100
        for k in range(3):
            est[j, k] = nowEstValues[k]

    return est, queryOrder

def get_truePr_RF(a, b, c, d):
    base = a + b + c + d
    truePr = [
        (a + c) / base,
        (a + b) / base,
        (b + d) / base,
        (c + d) / base,
        a / base,
        b / base,
        c / base,
        d / base,
        (a + b + c) / base,
        (a + b + d) / base,
        (a + c + d) / base,
        (b + c + d) / base,
        (a / (a + b)),
        (b / (a + b)),
        (c / (c + d)),
        (d / (c + d)),
        (a / (a + c)),
        (c / (a + c)),
        (b / (b + d)),
        (d / (b + d))
    ]
    return truePr

def generativeModel_RF(params):
    a, b, c, d = params
    truePr = get_truePr_RF(a, b, c, d)
    return truePr

def MSE_RF(params, data):
    predicted = generativeModel_RF(params)
    mse = np.mean((np.array(predicted) - data) ** 2)
    return mse

def MSE_RF(params, data):
    predicted = generativeModel_RF(params)
    mse = np.mean((predicted[4] - data) ** 2)
    return mse

participant_file = '/content/PrEstExp_811_111418_122039.csv'
data, queryOrder = clean_data_single_participant(participant_file)

target_data = data[4, :]
target_mean = np.mean(target_data)

fit_result = fit_RF_model(target_data)
mse = fit_result.fun
parameters = fit_result.x

b_0 = parameters[1]
prediction_cold_and_rainy = generativeModel_RF(parameters)[4]

print(f"MSE: {mse:.5f}")
print(f"b[0]: {b_0:.5f}")
print(f"Prediction for 'cold and rainy': {prediction_cold_and_rainy:.5f}")


MSE: 0.01556
b[0]: 10.87149
Prediction for 'cold and rainy': 0.66667
